In [ ]:
# Cài Java
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Tải và giải nén Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz

# Cài findspark để liên kết Spark với Colab
!pip install -q findspark


cai da spark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("HousePricePrediction").getOrCreate()


In [ ]:
try:
    from pyspark.sql import SparkSession

    spark = SparkSession.builder.appName("CheckSpark").getOrCreate()
    print("✅ Spark đã được cài!")
    print("🔍 Phiên bản Spark:", spark.version)
except Exception as e:
    print("❌ Spark chưa được cài.")
    print("Lỗi:", e)


✅ Spark đã được cài!
🔍 Phiên bản Spark: 3.5.1


In [ ]:
!java -version
!readlink -f $(which java)

openjdk version "11.0.27" 2025-04-15
OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
/usr/lib/jvm/java-11-openjdk-amd64/bin/java


In [1]:
try:
    import pyspark
    print("✅ PySpark đã được cài.")
except ImportError:
    print("❌ PySpark chưa được cài.")


✅ PySpark đã được cài.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder \
    .appName("MLlib Test") \
    .getOrCreate()

print(spark.version)


3.5.1


check


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("HousePricePrediction") \
    .getOrCreate()


In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="features", labelCol="label")


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MLlibTest").getOrCreate()
spark

import spark

In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

findspark.init()




Cai dat moi truong

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
csv_path = "/content/drive/MyDrive/Big Data/Project/data/HousePriceTraindatatest.csv"


In [ ]:
data = spark.read.csv(csv_path, header=True, inferSchema=True)
data.show(5)


+-------------------+---------+--------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+--------------------+---------+--------+-------+
|               date|    price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|sqft_above|sqft_basement|yr_built|yr_renovated|              street|     city|statezip|country|
+-------------------+---------+--------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+--------------------+---------+--------+-------+
|2014-05-02 00:00:00| 313000.0|     3.0|      1.5|       1340|    7912|   1.5|         0|   0|        3|      1340|            0|    1955|        2005|18810 Densmore Ave N|Shoreline|WA 98133|    USA|
|2014-05-02 00:00:00|2384000.0|     5.0|      2.5|       3650|    9050|   2.0|         0|   4|        5|      3370|          280|    1921|           0|     709 W Blaine St|  Seattle|WA 98119|    USA|


In [ ]:
data.printSchema()


root
 |-- date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: double (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- statezip: string (nullable = true)
 |-- country: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col

# Chọn các cột cần thiết
cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
        'view', 'condition', 'sqft_above', 'sqft_basement', 'yr_built',
        'yr_renovated', 'price']

# Ép kiểu dữ liệu sang số thực
data = data.select([col(c).cast("double") for c in cols])

# Xoá các dòng chứa giá trị null
data = data.dropna()


In [ ]:
from pyspark.ml.feature import VectorAssembler

feature_cols = [c for c in data.columns if c != 'price']
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

data = assembler.transform(data).select('features', 'price')
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)


In [ ]:
from pyspark.ml.regression import LinearRegression

# Tạo mô hình hồi quy tuyến tính
lr = LinearRegression(featuresCol='features', labelCol='price')

# Huấn luyện mô hình với tập train
lr_model = lr.fit(train_data)


In [ ]:
# Dự đoán trên tập test
predictions = lr_model.transform(test_data)
predictions.select("price", "prediction").show(5)


+--------+------------------+
|   price|        prediction|
+--------+------------------+
|276000.0| 288541.8034555251|
|299000.0|181040.08187181223|
|190000.0|328128.92167808395|
|129000.0| 236825.0029925732|
|     0.0|307894.53907257505|
+--------+------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

# Tạo bộ đánh giá
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")

# Tính toán các chỉ số
rmse = evaluator.setMetricName("rmse").evaluate(predictions)
mae = evaluator.setMetricName("mae").evaluate(predictions)
r2 = evaluator.setMetricName("r2").evaluate(predictions)

# In kết quả
print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ MAE: {mae:.2f}")
print(f"✅ R2: {r2:.4f}")


✅ RMSE: 229256.84
✅ MAE: 155462.02
✅ R2: 0.5057


In [ ]:
# Hiển thị hệ số và chệch
print("Hệ số (coefficients):", lr_model.coefficients)
print("Chệch (intercept):", lr_model.intercept)


Hệ số (coefficients): [-62942.235911520766,67394.52486084067,-2985.1855993470094,-0.70507718773557,24414.38452868245,54805.514202636456,30384.770278530894,3251.0768191587,3219.8779734690847,-2342.437296582884,6.509905399810215]
Chệch (intercept): 4531996.92381375


In [ ]:
from pyspark.ml.regression import RandomForestRegressor

# Tạo mô hình Random Forest
rf = RandomForestRegressor(featuresCol="features", labelCol="price", numTrees=100, maxDepth=10, seed=42)

# Huấn luyện mô hình
rf_model = rf.fit(train_data)


In [ ]:
# Dự đoán trên tập test
rf_predictions = rf_model.transform(test_data)
rf_predictions.select("price", "prediction").show(5)


+--------+------------------+
|   price|        prediction|
+--------+------------------+
|276000.0| 278359.6637395524|
|299000.0|266920.43134508276|
|190000.0| 304434.8979603931|
|129000.0|261578.90851653053|
|     0.0| 274759.8880366312|
+--------+------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

# Tạo bộ đánh giá
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")

# Tính các chỉ số đánh giá
rf_rmse = evaluator.setMetricName("rmse").evaluate(rf_predictions)
rf_mae = evaluator.setMetricName("mae").evaluate(rf_predictions)
rf_r2 = evaluator.setMetricName("r2").evaluate(rf_predictions)

# In kết quả
print(f"🌲 Random Forest RMSE: {rf_rmse:.2f}")
print(f"🌲 Random Forest MAE: {rf_mae:.2f}")
print(f"🌲 Random Forest R2: {rf_r2:.4f}")


🌲 Random Forest RMSE: 260588.23
🌲 Random Forest MAE: 165450.60
🌲 Random Forest R2: 0.3614


In [ ]:
# Đọc lại dữ liệu từ file CSV gốc
data = spark.read.csv(csv_path, header=True, inferSchema=True)


In [ ]:
from pyspark.sql.functions import col

cat_cols = ['city', 'statezip']
num_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
            'view', 'condition', 'sqft_above', 'sqft_basement', 'yr_built',
            'yr_renovated']
label_col = 'price'

# Lấy đúng cột và ép kiểu số cho numeric
data = data.select(
    [col(c).cast("double") if c in num_cols + [label_col] else col(c) for c in num_cols + cat_cols + [label_col]]
).dropna()


In [ ]:
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=c, outputCol=c+"_idx") for c in cat_cols]


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor

# Gộp cột đặc trưng
feature_cols = num_cols + [c + "_idx" for c in cat_cols]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
# Mô hình Random Forest đã chỉnh maxBins
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="price",
    numTrees=100,
    maxDepth=10,
    maxBins=128,
    seed=42
)

# Pipeline gồm: mã hóa + vector hóa + mô hình
pipeline = Pipeline(stages=indexers + [assembler, rf])

# Tách tập train/test
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Huấn luyện mô hình
rf_model = pipeline.fit(train_data)

# Dự đoán
predictions = rf_model.transform(test_data)


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")

rmse = evaluator.setMetricName("rmse").evaluate(predictions)
mae = evaluator.setMetricName("mae").evaluate(predictions)
r2 = evaluator.setMetricName("r2").evaluate(predictions)

print(f"🏡 Random Forest + City/Zip RMSE: {rmse:.2f}")
print(f"🏡 Random Forest + City/Zip MAE: {mae:.2f}")
print(f"🏡 Random Forest + City/Zip R2: {r2:.4f}")


🏡 Random Forest + City/Zip RMSE: 232481.79
🏡 Random Forest + City/Zip MAE: 120134.04
🏡 Random Forest + City/Zip R2: 0.4917


In [ ]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

# Tạo lại indexer và assembler
indexers = [StringIndexer(inputCol=c, outputCol=c + "_idx") for c in ['city', 'statezip']]
feature_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                'view', 'condition', 'sqft_above', 'sqft_basement', 'yr_built',
                'yr_renovated', 'city_idx', 'statezip_idx']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Bộ đánh giá
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")

# Danh sách mô hình cần thử
param_sets = [
    {"numTrees": 100, "maxDepth": 10, "maxBins": 128},
    {"numTrees": 200, "maxDepth": 15, "maxBins": 128},
    {"numTrees": 300, "maxDepth": 20, "maxBins": 256},
]

for i, params in enumerate(param_sets, start=1):
    print(f"\n🔍 Đang chạy mô hình cấu hình {i}: {params}")

    rf = RandomForestRegressor(
        featuresCol="features",
        labelCol="price",
        numTrees=params["numTrees"],
        maxDepth=params["maxDepth"],
        maxBins=params["maxBins"],
        seed=42
    )

    pipeline = Pipeline(stages=indexers + [assembler, rf])
    model = pipeline.fit(train_data)
    predictions = model.transform(test_data)

    rmse = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse").evaluate(predictions)
    mae = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae").evaluate(predictions)
    r2 = evaluator.evaluate(predictions)

    print(f"🧪 Config {i} ➤ RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.4f}")



🔍 Đang chạy mô hình cấu hình 1: {'numTrees': 100, 'maxDepth': 10, 'maxBins': 128}
🧪 Config 1 ➤ RMSE: 232481.79, MAE: 120134.04, R²: 0.4917

🔍 Đang chạy mô hình cấu hình 2: {'numTrees': 200, 'maxDepth': 15, 'maxBins': 128}
🧪 Config 2 ➤ RMSE: 230352.25, MAE: 119988.40, R²: 0.5009

🔍 Đang chạy mô hình cấu hình 3: {'numTrees': 300, 'maxDepth': 20, 'maxBins': 256}
🧪 Config 3 ➤ RMSE: 222681.04, MAE: 118275.83, R²: 0.5336


In [ ]:
from pyspark.sql.functions import col, when, year, to_date
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

# Bước 1: Đọc lại dữ liệu gốc
data = spark.read.csv(csv_path, header=True, inferSchema=True)

# Bước 2: Tạo đặc trưng mới
data = data.withColumn("age_of_house", 2025 - col("yr_built"))
data = data.withColumn("was_renovated", when(col("yr_renovated") > 0, 1).otherwise(0))
data = data.withColumn("total_area", col("sqft_living") + col("sqft_basement"))

# Ép kiểu các cột số
numeric_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                'view', 'condition', 'sqft_above', 'sqft_basement', 'yr_built',
                'yr_renovated', 'age_of_house', 'was_renovated', 'total_area', 'price']

for c in numeric_cols:
    data = data.withColumn(c, col(c).cast("double"))

# Bước 3: Mã hóa biến phân loại
cat_cols = ['city', 'statezip']
indexers = [StringIndexer(inputCol=c, outputCol=c + "_idx") for c in cat_cols]

# Tạo danh sách cột đặc trưng đầu vào
feature_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                'view', 'condition', 'sqft_above', 'sqft_basement', 'age_of_house',
                'was_renovated', 'total_area', 'city_idx', 'statezip_idx']

# VectorAssembler
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Bước 4: Tạo mô hình GBT
gbt = GBTRegressor(featuresCol="features", labelCol="price", maxIter=200, maxDepth=7, maxBins=256)

# Tạo pipeline tổng thể
pipeline = Pipeline(stages=indexers + [assembler, gbt])

# Bước 5: Xử lý thiếu dữ liệu và chia train/test
data = data.dropna()
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Huấn luyện mô hình
gbt_model = pipeline.fit(train_data)

# Dự đoán
predictions = gbt_model.transform(test_data)


In [ ]:
# Trước đây (lỗi):
# indexers = [StringIndexer(inputCol=c, outputCol=c + "_idx") for c in ['city', 'statezip']]

# Sửa lại:
indexers = [
    StringIndexer(inputCol=c, outputCol=c + "_idx", handleInvalid="keep")
    for c in ['city', 'statezip']
]


In [ ]:
data = spark.read.csv(csv_path, header=True, inferSchema=True)


In [ ]:
from pyspark.sql.functions import col, when

data = data.withColumn("age_of_house", 2025 - col("yr_built"))
data = data.withColumn("was_renovated", when(col("yr_renovated") > 0, 1).otherwise(0))
data = data.withColumn("total_area", col("sqft_living") + col("sqft_basement"))


In [ ]:
numeric_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                'view', 'condition', 'sqft_above', 'sqft_basement', 'yr_built',
                'yr_renovated', 'age_of_house', 'was_renovated', 'total_area', 'price']

for c in numeric_cols:
    data = data.withColumn(c, col(c).cast("double"))

data = data.dropna()


In [ ]:
from pyspark.ml.feature import StringIndexer

cat_cols = ['city', 'statezip']
indexers = [
    StringIndexer(inputCol=c, outputCol=c + "_idx", handleInvalid="keep")
    for c in cat_cols
]


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline

feature_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                'view', 'condition', 'sqft_above', 'sqft_basement', 'age_of_house',
                'was_renovated', 'total_area', 'city_idx', 'statezip_idx']

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

gbt = GBTRegressor(featuresCol="features", labelCol="price", maxIter=200, maxDepth=7, maxBins=256)

pipeline = Pipeline(stages=indexers + [assembler, gbt])


In [ ]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
gbt_model = pipeline.fit(train_data)
predictions = gbt_model.transform(test_data)


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")
rmse = evaluator.setMetricName("rmse").evaluate(predictions)
mae = evaluator.setMetricName("mae").evaluate(predictions)
r2 = evaluator.setMetricName("r2").evaluate(predictions)

print(f"GBTRegressor RMSE: {rmse:.2f}")
print(f"GBTRegressor MAE: {mae:.2f}")
print(f"GBTRegressor R²: {r2:.4f}")


GBTRegressor RMSE: 1036064.87
GBTRegressor MAE: 175768.25
GBTRegressor R²: 0.0231


In [ ]:
from pyspark.sql.functions import log1p

# Tạo biến mới log_price = log(1 + price)
data = data.withColumn("log_price", log1p(col("price")))


In [ ]:
gbt = GBTRegressor(featuresCol="features", labelCol="log_price", maxIter=200, maxDepth=7, maxBins=256)


In [ ]:
from pyspark.sql.functions import expm1

# Dự đoán xong rồi:
predictions = gbt_model.transform(test_data)

# Chuyển từ log(price) về lại price
predictions = predictions.withColumn("prediction_price", expm1(col("prediction")))


In [ ]:
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction_price")
rmse = evaluator.setMetricName("rmse").evaluate(predictions)
mae = evaluator.setMetricName("mae").evaluate(predictions)
r2 = evaluator.setMetricName("r2").evaluate(predictions)

print(f" GBT (log-price) RMSE: {rmse:.2f}")
print(f" GBT (log-price) MAE: {mae:.2f}")
print(f" GBT (log-price) R²: {r2:.4f}")


 GBT (log-price) RMSE: inf
 GBT (log-price) MAE: inf
 GBT (log-price) R²: -inf


In [ ]:
# Chỉ giữ các dòng có price > 0
data = data.filter(col("price") > 0)

# Tạo log_price sau khi lọc
from pyspark.sql.functions import log1p
data = data.withColumn("log_price", log1p(col("price")))


In [ ]:
from pyspark.sql.functions import expm1

predictions = gbt_model.transform(test_data)
predictions = predictions.withColumn("prediction_price", expm1(col("prediction")))


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction_price")
rmse = evaluator.setMetricName("rmse").evaluate(predictions)
mae = evaluator.setMetricName("mae").evaluate(predictions)
r2 = evaluator.setMetricName("r2").evaluate(predictions)

print(f"✅ GBT (log-price, clean) RMSE: {rmse:.2f}")
print(f"✅ GBT (log-price, clean) MAE: {mae:.2f}")
print(f"✅ GBT (log-price, clean) R²: {r2:.4f}")


✅ GBT (log-price, clean) RMSE: inf
✅ GBT (log-price, clean) MAE: inf
✅ GBT (log-price, clean) R²: -inf


In [ ]:
predictions.select("prediction", "price").show(10)


+------------------+--------+
|        prediction|   price|
+------------------+--------+
|245468.95266441183|242500.0|
| 378104.7593569519|284000.0|
| 389074.3013421495|287200.0|
|324690.35468439764|315000.0|
| 433643.0973688922|382500.0|
|422664.36640199134|407500.0|
|502946.56333111867|437500.0|
|450136.51794225466|459990.0|
| 647791.0334011437|555000.0|
| 652015.5432094008|588500.0|
+------------------+--------+
only showing top 10 rows



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")

rmse = evaluator.setMetricName("rmse").evaluate(predictions)
mae = evaluator.setMetricName("mae").evaluate(predictions)
r2 = evaluator.setMetricName("r2").evaluate(predictions)

print(f"✅ GBT (giá gốc) RMSE: {rmse:.2f}")
print(f"✅ GBT (giá gốc) MAE: {mae:.2f}")
print(f"✅ GBT (giá gốc) R²: {r2:.4f}")


✅ GBT (giá gốc) RMSE: 1036064.87
✅ GBT (giá gốc) MAE: 175768.25
✅ GBT (giá gốc) R²: 0.0231


In [ ]:
# Giả sử rf_model là mô hình Random Forest tốt nhất đã huấn luyện (config 3)
rf_model.save("/content/best_rf_model")


In [ ]:
# Nén thư mục model
!zip -r best_rf_model.zip best_rf_model

# Tải file zip về
from google.colab import files
files.download("best_rf_model.zip")


  adding: best_rf_model/ (stored 0%)
  adding: best_rf_model/stages/ (stored 0%)
  adding: best_rf_model/stages/3_RandomForestRegressor_0238ed250c3a/ (stored 0%)
  adding: best_rf_model/stages/3_RandomForestRegressor_0238ed250c3a/treesMetadata/ (stored 0%)
  adding: best_rf_model/stages/3_RandomForestRegressor_0238ed250c3a/treesMetadata/.part-00000-9b9e97b7-6524-4184-85b1-9c4a34a26c7c-c000.snappy.parquet.crc (stored 0%)
  adding: best_rf_model/stages/3_RandomForestRegressor_0238ed250c3a/treesMetadata/part-00000-9b9e97b7-6524-4184-85b1-9c4a34a26c7c-c000.snappy.parquet (deflated 25%)
  adding: best_rf_model/stages/3_RandomForestRegressor_0238ed250c3a/treesMetadata/_SUCCESS (stored 0%)
  adding: best_rf_model/stages/3_RandomForestRegressor_0238ed250c3a/treesMetadata/._SUCCESS.crc (stored 0%)
  adding: best_rf_model/stages/3_RandomForestRegressor_0238ed250c3a/data/ (stored 0%)
  adding: best_rf_model/stages/3_RandomForestRegressor_0238ed250c3a/data/part-00000-3eeb13e3-a986-4f4a-af2e-31c5ba

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data = spark.read.csv(csv_path, header=True, inferSchema=True)


In [ ]:
from pyspark.sql.functions import col, when

# Đặc trưng mở rộng
data = data.withColumn("age_of_house", 2025 - col("yr_built"))
data = data.withColumn("was_renovated", when(col("yr_renovated") > 0, 1).otherwise(0))
data = data.withColumn("total_area", col("sqft_living") + col("sqft_basement"))

# Ép kiểu cho các cột số
num_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
            'view', 'condition', 'sqft_above', 'sqft_basement', 'yr_built',
            'yr_renovated', 'age_of_house', 'was_renovated', 'total_area', 'price']

for c in num_cols:
    data = data.withColumn(c, col(c).cast("double"))

data = data.dropna()


In [ ]:
from pyspark.ml.feature import StringIndexer

indexers = [
    StringIndexer(inputCol=c, outputCol=c + "_idx", handleInvalid="keep")
    for c in ['city', 'statezip']
]


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline

feature_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                'view', 'condition', 'sqft_above', 'sqft_basement', 'age_of_house',
                'was_renovated', 'total_area', 'city_idx', 'statezip_idx']

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="price",
    numTrees=300,
    maxDepth=20,
    maxBins=256,
    seed=42
)

pipeline = Pipeline(stages=indexers + [assembler, rf])


In [ ]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
rf_model = pipeline.fit(train_data)
predictions = rf_model.transform(test_data)


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")
rmse = evaluator.setMetricName("rmse").evaluate(predictions)
mae = evaluator.setMetricName("mae").evaluate(predictions)
r2 = evaluator.setMetricName("r2").evaluate(predictions)

print(f"✅ Retrained RF RMSE: {rmse:.2f}")
print(f"✅ Retrained RF MAE: {mae:.2f}")
print(f"✅ Retrained RF R²: {r2:.4f}")


✅ Retrained RF RMSE: 1007454.93
✅ Retrained RF MAE: 153601.33
✅ Retrained RF R²: 0.0763


In [ ]:
from pyspark.sql.functions import col, when
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

# Đọc dữ liệu gốc
data = spark.read.csv(csv_path, header=True, inferSchema=True)

# Tạo đặc trưng mới
data = data.withColumn("age_of_house", 2025 - col("yr_built"))
data = data.withColumn("was_renovated", when(col("yr_renovated") > 0, 1).otherwise(0))
data = data.withColumn("total_area", col("sqft_living") + col("sqft_basement"))

# Ép kiểu số
num_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
            'view', 'condition', 'sqft_above', 'sqft_basement', 'yr_built',
            'yr_renovated', 'age_of_house', 'was_renovated', 'total_area', 'price']

for c in num_cols:
    data = data.withColumn(c, col(c).cast("double"))

# Xử lý thiếu
data = data.dropna()

# Mã hóa các biến phân loại
indexers = [
    StringIndexer(inputCol=c, outputCol=c + "_idx", handleInvalid="keep")
    for c in ['city', 'statezip']
]

# Gộp các đặc trưng đầu vào
feature_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                'view', 'condition', 'sqft_above', 'sqft_basement', 'age_of_house',
                'was_renovated', 'total_area', 'city_idx', 'statezip_idx']

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Tạo mô hình Random Forest tốt nhất
rf = RandomForestRegressor(
    featuresCol="features", labelCol="price",
    numTrees=300, maxDepth=20, maxBins=256, seed=42
)

# Pipeline tổng hợp
pipeline = Pipeline(stages=indexers + [assembler, rf])

# Chia tập train/test
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Huấn luyện mô hình
rf_model = pipeline.fit(train_data)

# Dự đoán
predictions = rf_model.transform(test_data)

# Đánh giá mô hình
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")
rmse = evaluator.setMetricName("rmse").evaluate(predictions)
mae = evaluator.setMetricName("mae").evaluate(predictions)
r2 = evaluator.setMetricName("r2").evaluate(predictions)

print(f"✅ RF (tốt nhất) RMSE: {rmse:.2f}")
print(f"✅ RF (tốt nhất) MAE: {mae:.2f}")
print(f"✅ RF (tốt nhất) R²: {r2:.4f}")


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JJavaError: An error occurred while calling o45731.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6072.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6072.0 (TID 6052) (357c9aacbe54 executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGSche

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("HousePricePrediction") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()
